In [40]:
#pip install pandas numpy matplotlib tensorflow scikit-learn pandas_datareader

In [41]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Carregar os dados
df = pd.read_csv('C:\\Users\\denis\\Estudos\\Estudos_IBM\\Certificado Profissional Engenharia de IA da IBM\\Introdução à aprendizagem profunda e às redes neurais com o Keras\\Dados Históricos - Shiba Inu.csv')

# Converter a coluna de data para datetime
df['Data'] = pd.to_datetime(df['Data'], format='%d.%m.%Y')

# Substituir vírgulas por pontos e converter as colunas para float
for feature in ['Último', 'Máxima', 'Mínima']:
    df[feature] = df[feature].str.replace(',', '.').astype(float)

# Definir as features e preparar os dados
features = ['Último', 'Máxima', 'Mínima']

# Normalizar dados
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df[features])

# Preparar dados para treinamento
def create_sequences(data, time_step=10):
    X, y = [], []
    for i in range(len(data) - time_step):
        X.append(data[i:i + time_step])
        y.append(data[i + time_step])
    return np.array(X), np.array(y)

time_step = 10  # Número de passos no tempo
X, y = create_sequences(scaled_data, time_step)

# Dividir os dados em treinamento e teste
split_index = int(0.8 * len(X))
X_train, X_test = X[:split_index], X[split_index:]
y_train, y_test = y[:split_index], y[split_index:]

# Construir o modelo
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(time_step, len(features))),
    LSTM(50),
    Dense(len(features))  # Prever todas as features
])

model.compile(optimizer='adam', loss='mean_squared_error')

# Treinar o modelo
history = model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=32,
    validation_split=0.1
)

# Avaliar o modelo
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)

# Inverter a normalização das previsões
train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)

# Calcular métricas de erro
mae_train = mean_absolute_error(scaler.inverse_transform(y_train), train_predict)
mse_train = mean_squared_error(scaler.inverse_transform(y_train), train_predict)
rmse_train = np.sqrt(mse_train)
r2_train = r2_score(scaler.inverse_transform(y_train), train_predict)

mae_test = mean_absolute_error(scaler.inverse_transform(y_test), test_predict)
mse_test = mean_squared_error(scaler.inverse_transform(y_test), test_predict)
rmse_test = np.sqrt(mse_test)
r2_test = r2_score(scaler.inverse_transform(y_test), test_predict)

# Exibir resultados
print(f"MAE (Treinamento): {mae_train}")
print(f"MSE (Treinamento): {mse_train}")
print(f"RMSE (Treinamento): {rmse_train}")
print(f"R² (Treinamento): {r2_train}")

print(f"MAE (Teste): {mae_test}")
print(f"MSE (Teste): {mse_test}")
print(f"RMSE (Teste): {rmse_test}")
print(f"R² (Teste): {r2_test}")

# Prever futuros valores
def predict_future(model, last_data, future_steps, scaler, time_step):
    predictions = []
    data = last_data.copy()
    for _ in range(future_steps):
        pred = model.predict(data[np.newaxis, ...])
        predictions.append(pred[0])
        data = np.vstack([data[1:], pred])
    return np.array(predictions)

# Dados para previsão futura
last_data = scaled_data[-time_step:]
future_steps = 30
future_predictions = predict_future(model, last_data, future_steps, scaler, time_step)

# Inverter a normalização dos dados futuros
future_predictions = scaler.inverse_transform(future_predictions)

# Configurar a exibição de números no formato decimal completo
np.set_printoptions(precision=10, suppress=True)

# Agora, as previsões futuras serão exibidas sem notação científica
print(f"Previsões futuras para {future_steps} dias:")
print(future_predictions)




Epoch 1/100


C:\Users\denis\anaconda3\envs\IA_env\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


26/26 ━━━━━━━━━━━━━━━━━━━━ 4s 28ms/step - loss: 0.0071 - val_loss: 0.0029
Epoch 2/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 8.4304e-04 - val_loss: 0.0020
Epoch 3/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 9.4901e-04 - val_loss: 0.0020
Epoch 4/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 7.0113e-04 - val_loss: 0.0018
Epoch 5/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 6.8526e-04 - val_loss: 0.0018
Epoch 6/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 7.0094e-04 - val_loss: 0.0017
Epoch 7/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 4.7838e-04 - val_loss: 0.0015
Epoch 8/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 5.8472e-04 - val_loss: 0.0014
Epoch 9/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 5.4510e-04 - val_loss: 0.0014
Epoch 10/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 5.3505e-04 - val_loss: 0.0012
Epoch 11/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 4.6793e-04 - val_loss: 0.0020
Epoch 12/100
26/26 ━━━━━━━━━━

In [43]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Carregar os dados
df = pd.read_csv('C:\\Users\\denis\\Estudos\\Estudos_IBM\\Certificado Profissional Engenharia de IA da IBM\\Introdução à aprendizagem profunda e às redes neurais com o Keras\\Dados Históricos - Shiba Inu.csv')

# Converter a coluna de data para datetime
df['Data'] = pd.to_datetime(df['Data'], format='%d.%m.%Y')

# Substituir vírgulas por pontos e converter as colunas para float
for feature in ['Último', 'Máxima', 'Mínima']:
    df[feature] = df[feature].str.replace(',', '.').astype(float)

# Definir as features e preparar os dados
features = ['Último', 'Máxima', 'Mínima']

# Normalizar dados
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df[features])

# Preparar dados para treinamento
def create_sequences(data, time_step=10):
    X, y = [], []
    for i in range(len(data) - time_step):
        X.append(data[i:i + time_step])
        y.append(data[i + time_step])
    return np.array(X), np.array(y)

time_step = 10  # Número de passos no tempo
X, y = create_sequences(scaled_data, time_step)

# Dividir os dados em treinamento e teste
split_index = int(0.8 * len(X))
X_train, X_test = X[:split_index], X[split_index:]
y_train, y_test = y[:split_index], y[split_index:]

# Construir o modelo
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(time_step, len(features))),
    LSTM(50),
    Dense(len(features))  # Prever todas as features
])

model.compile(optimizer='adam', loss='mean_squared_error')

# Treinar o modelo
history = model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=32,
    validation_split=0.1
)

# Avaliar o modelo
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)

# Inverter a normalização das previsões
train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)

# Calcular métricas de erro
mae_train = mean_absolute_error(scaler.inverse_transform(y_train), train_predict)
mse_train = mean_squared_error(scaler.inverse_transform(y_train), train_predict)
rmse_train = np.sqrt(mse_train)
r2_train = r2_score(scaler.inverse_transform(y_train), train_predict)

mae_test = mean_absolute_error(scaler.inverse_transform(y_test), test_predict)
mse_test = mean_squared_error(scaler.inverse_transform(y_test), test_predict)
rmse_test = np.sqrt(mse_test)
r2_test = r2_score(scaler.inverse_transform(y_test), test_predict)

# Exibir resultados
print(f"MAE (Treinamento): {mae_train}")
print(f"MSE (Treinamento): {mse_train}")
print(f"RMSE (Treinamento): {rmse_train}")
print(f"R² (Treinamento): {r2_train}")

print(f"MAE (Teste): {mae_test}")
print(f"MSE (Teste): {mse_test}")
print(f"RMSE (Teste): {rmse_test}")
print(f"R² (Teste): {r2_test}")

# Prever futuros valores
def predict_future(model, last_data, future_steps, scaler, time_step):
    predictions = []
    data = last_data.copy()
    for _ in range(future_steps):
        pred = model.predict(data[np.newaxis, ...])
        predictions.append(pred[0])
        data = np.vstack([data[1:], pred])
    return np.array(predictions)

# Dados para previsão futura
last_data = scaled_data[-time_step:]
future_steps = 30
future_predictions = predict_future(model, last_data, future_steps, scaler, time_step)

# Configurar a exibição de números no formato decimal completo
np.set_printoptions(precision=10, suppress=True)

# Exibir previsões futuras com numeração das linhas e colunas
print(f"Previsões futuras para {future_steps} dias:")
for i, prediction in enumerate(future_predictions):
    print(f"Dia {i+1}:")
    print(f"  Último: {prediction[0]:.10f}")
    print(f"  Máxima: {prediction[1]:.10f}")
    print(f"  Mínima: {prediction[2]:.10f}")


C:\Users\denis\anaconda3\envs\IA_env\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 5s 30ms/step - loss: 0.0073 - val_loss: 0.0043
Epoch 2/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 8.5956e-04 - val_loss: 0.0021
Epoch 3/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 9.4639e-04 - val_loss: 0.0022
Epoch 4/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 6.6679e-04 - val_loss: 0.0022
Epoch 5/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 6.3485e-04 - val_loss: 0.0019
Epoch 6/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 5.8441e-04 - val_loss: 0.0018
Epoch 7/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 5.5651e-04 - val_loss: 0.0020
Epoch 8/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 4.8702e-04 - val_loss: 0.0016
Epoch 9/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 5.6129e-04 - val_loss: 0.0016
Epoch 10/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 3.7915e-04 - val_loss: 0.0012
Epoch 11/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 3.4029e-04 - val_loss: 0.0013
Epoch 12/100
2